# 第8章: 機械学習

In [36]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re

## 70. データの入手・整形
文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [4]:
cd rt-polaritydata/

/Users/fukuta-mil/Program/python/Misc/Practice/100knock/rt-polaritydata/rt-polaritydata


In [193]:
import random
with open('rt-polarity.pos', encoding='latin_1') as f1, open('rt-polarity.neg', encoding='latin_1') as f2, open('sentiment.txt', 'w') as f:
    text = []
    for line in f1:
        text.append("+1 "+line)
    for line in f2:
        text.append("-1 "+line)
    random.shuffle(text)
    f.write(''.join(text))

## 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [63]:
class Util:
    stop_words = get_stop_words('english')

    @staticmethod
    def is_stop_word(_word):
        return _word in Util.stop_words

In [70]:
Util.is_stop_word('he')    

True

## 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [215]:
with open('sentiment.txt') as f:
    text = f.read().split('\n')[:-1]

fig = re.compile(r'[,.:;/\'\"\-\(\)!]+')
space = re.compile(r'\s+')
stemmer = PorterStemmer()

data = []

for line in text:
    feature = []
    label = line[:3]
    x = space.split(fig.sub(' ',line[3:]))
    for word in x:
        stem = stemmer.stem(word)
        if Util.is_stop_word(stem) or word=='':
            continue
        feature.append(stem)
    data.append((label,feature))
    
from functools import reduce

vocab = set(reduce(lambda a,b: a+b, [x[1] for x in data]))
special_words = set(['<unk>'])

word2index = dict(map(lambda x: (x[1], x[0]), enumerate(vocab | special_words)))
index2word = list(vocab | special_words)

x_data = []
y_data = []
for label, s in data:
    x_data.append([word2index[w] for w in s])
    y_data.append((int(label)+1)//2)

## 73. 学習
72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [217]:
X_data = np.zeros((len(x_data), len(word2index)))
for i in range(len(x_data)):
    for j in x_data[i]:
        X_data[i][j] = 1

In [221]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(penalty='l1')
clf_lr.fit(X_data[100:], y_data[100:])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [234]:
from sklearn.metrics import f1_score
f1_score(clf_lr.predict(X_data[:100]), y_data[:100])

0.76923076923076916

array([12846,  7188,  2794,  1024,  6623,  5319,  3202,  5456, 11677,  9572])

In [267]:
list(zip(np.array(index2word)[np.argsort(clf_lr.coef_[0])[::-1][:10]], sorted(clf_lr.coef_[0])[::-1][:10]))

[('engross', 3.1536107264284028),
 ('resist', 2.5688042846464367),
 ('refresh', 2.494846045429433),
 ('unexpect', 2.475847314158369),
 ('acknowledg', 2.4368640250080125),
 ('sli', 2.3435443264266755),
 ('gloriou', 2.3313605598270213),
 ('smarter', 2.228294159787132),
 ('son', 2.136560781418249),
 ('outlandish', 2.0846022576158787)]